<a href="https://colab.research.google.com/github/AzeemWaqarRao/Pytorch_Implementations/blob/main/PytorchRNNLayer(MNIST_Problem).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
import torchvision
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

In [ ]:
# hyperparameters

batch_size = 32
input_len = 28
sequence_size = 28
# image is 28*28 and we will feed it row by row

num_classes = 10
num_layers = 2
hidden_size = 128
learning_rate = 0.0005
epochs = 10

In [ ]:
train_data = MNIST('root',train = True,transform=ToTensor(),download=True)
test_data = MNIST('root',train = False,transform=ToTensor(),download=True)

In [ ]:
train_data = DataLoader(train_data,batch_size,shuffle = True)
test_data = DataLoader(test_data,batch_size,shuffle = True)

In [ ]:
class RNN(nn.Module):
  def __init__(self,input_size,hidden_size,num_layers,output_size):
    super().__init__()
    self.num_layers = num_layers
    self.hidden_size = hidden_size
    self.rnn = nn.RNN(input_size,hidden_size,num_layers,batch_first = True)
    self.linear = nn.Linear(self.hidden_size,output_size)

  def forward(self,x):
    h_0 = torch.zeros(self.num_layers,x.size(0),self.hidden_size)
    out, _ = self.rnn(x,h_0) # size(out) : batch_size, sequence_len, hidden_state : N, 28, 128
    out = out[:,-1,:] # size(out) : batch_size, hidden_state
    out = self.linear(out)
    return out

In [ ]:
rnn = RNN(input_len,hidden_size,num_layers,num_classes)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(epochs):
  for image, label in train_data:
    image = image.view(-1,28,28)
    y_pred = rnn(image)
    loss = loss_fn(y_pred,label)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(f"loss {loss.item()}")

Streaming output truncated to the last 5000 lines.
loss 0.01535253319889307
loss 0.19063816964626312
loss 0.0838589072227478
loss 0.16373859345912933
loss 0.022654132917523384
loss 0.22072234749794006
loss 0.008788857609033585
loss 0.01277440320700407
loss 0.20072419941425323
loss 0.14869996905326843
loss 0.30201029777526855
loss 0.02181258797645569
loss 0.25349685549736023
loss 0.09418826550245285
loss 0.12270691245794296
loss 0.038237638771533966
loss 0.13539855182170868
loss 0.008546652272343636
loss 0.014658675529062748
loss 0.008914071135222912
loss 0.04991956800222397
loss 0.106782928109169
loss 0.018718337640166283
loss 0.053109269589185715
loss 0.03669572249054909
loss 0.08845174312591553
loss 0.22162093222141266
loss 0.17327852547168732
loss 0.12735611200332642
loss 0.015913179144263268
loss 0.0911945253610611
loss 0.007333741057664156
loss 0.36872580647468567
loss 0.07838848978281021
loss 0.2879253625869751
loss 0.13362501561641693
loss 0.06055532023310661
loss 0.147195383906

In [ ]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_data:
        images = images.reshape(-1, sequence_size, input_len)
        outputs = rnn(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the 10000 test images: {acc} %')

Accuracy of the network on the 10000 test images: 97.49 %
